In [1]:
import pandas as pd
from finta import TA
import numpy as np

In [2]:
#yahoo finance stock data (for longer timeframe)
import yfinance as yf

def stock_df(ticker, start, end):
    stock = yf.Ticker(ticker)
    stock_df = stock.history(start = start, end = end)
    return stock_df

start = pd.to_datetime('2015-01-01')
end = pd.to_datetime('today')
                     
spy_df = stock_df('SPY', start, end)

len(spy_df)

1770

In [3]:
# spy_df["Monetary Gain"] = spy_df["Close"].diff()
spy_df['Actual Return'] = spy_df["Close"].pct_change()

spy_df.loc[(spy_df['Actual Return'] >= 0), 'Return Direction'] = 1
spy_df.loc[(spy_df['Actual Return'] < 0), 'Return Direction'] = 0

# spy_df['Trades'] = np.abs(spy_df['Trading Signal'].diff())

# spy_df['Strategy Returns'] = spy_df['Actual Return'] * spy_df['Trading Signal'].shift()

spy_df.dropna(inplace= True)

spy_df.tail(20)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Actual Return,Return Direction
Date,,,,,,,,,
2021-12-14,461.468749,464.109466,458.638695,461.737793,97264100,0.000,0,-0.006880,0.0
2021-12-15,461.797598,469.211523,459.126957,468.952454,116899300,0.000,0,0.015625,1.0
2021-12-16,470.915555,471.214493,463.172738,464.816986,116568600,0.000,0,-0.008819,0.0
2021-12-17,461.549988,464.739990,458.059998,459.869995,135511600,1.633,0,-0.010643,0.0
2021-12-20,454.480011,455.399994,451.140015,454.980011,107134800,0.000,0,-0.010633,0.0
2021-12-21,458.609985,463.209991,456.309998,463.059998,69806300,0.000,0,0.017759,1.0
2021-12-22,462.790009,467.809998,462.579987,467.690002,58890200,0.000,0,0.009999,1.0
2021-12-23,468.750000,472.190002,468.640015,470.600006,56439700,0.000,0,0.006222,1.0
2021-12-27,472.059998,477.309998,472.010010,477.260010,56808600,0.000,0,0.014152,1.0


In [4]:
spy_technical_indicators = pd.DataFrame()

#Creating RSI evaluation metrics    
spy_technical_indicators["RSI"] = TA.RSI(spy_df, 14)
spy_technical_indicators["RSI Evaluation"] = "No Abnormalities"
spy_technical_indicators.loc[spy_technical_indicators["RSI"] > 70, 'RSI Evaluation'] = "Overvalued"
spy_technical_indicators.loc[spy_technical_indicators["RSI"] < 30, 'RSI Evaluation'] = "Undervalued"

spy_technical_indicators["RSI Lag"] = spy_technical_indicators["RSI Evaluation"].shift(1)


for index, row in spy_technical_indicators.iterrows():
    if (spy_technical_indicators.loc[index, "RSI Evaluation"] == "No Abnormalities" and spy_technical_indicators.loc[index, "RSI Lag"] == "Undervalued"):
            spy_technical_indicators.loc[index, "RSI Evaluation"] = "RSI Bullish Signal"
    if (spy_technical_indicators.loc[index, "RSI Evaluation"] == "No Abnormalities" and spy_technical_indicators.loc[index, "RSI Lag"] == "Overvalued"):
            spy_technical_indicators.loc[index, "RSI Evaluation"] = "RSI Bearish Signal"


spy_technical_indicators.drop(columns = ["RSI Lag"], inplace = True)
         
#Creating CCI evaluation metrics    
spy_technical_indicators["CCI"] = TA.CCI(spy_df, 14)
spy_technical_indicators["CCI Lag"] = spy_technical_indicators["CCI"].shift(1)

for index, row in spy_technical_indicators.iterrows():
    if (spy_technical_indicators.loc[index, "CCI"] > spy_technical_indicators.loc[index, "CCI Lag"]):
        if (spy_technical_indicators.loc[index, "CCI"] > 0 and spy_technical_indicators.loc[index, "CCI Lag"] < 0):
            spy_technical_indicators.loc[index, "CCI Evaluation"] = "CCI Uptrend"
        else:
            spy_technical_indicators.loc[index, "CCI Evaluation"] = "CCI Potential Uptrend"
            
    if (spy_technical_indicators.loc[index, "CCI"] < spy_technical_indicators.loc[index, "CCI Lag"]):
        if (spy_technical_indicators.loc[index, "CCI"] < 0 and spy_technical_indicators.loc[index, "CCI Lag"] > 0):
            spy_technical_indicators.loc[index, "CCI Evaluation"] = "CCI Downtrend"
        else:
            spy_technical_indicators.loc[index, "CCI Evaluation"] = "CCI Potential Downtrend"

spy_technical_indicators.drop(columns = ["CCI Lag"], inplace = True)

#Creating Rate of Change evaluation metrics
spy_technical_indicators["ROC"] = TA.ROC(spy_df, 20)
spy_technical_indicators["ROC Lag"] = spy_technical_indicators["ROC"].shift(1)

for index, row in spy_technical_indicators.iterrows():
    if (spy_technical_indicators.loc[index, "ROC"] > 0 and spy_technical_indicators.loc[index, "ROC Lag"] < 0):
            spy_technical_indicators.loc[index, "ROC Evaluation"] = "ROC Rising Momentum"
    if (spy_technical_indicators.loc[index, "ROC"] < 0 and spy_technical_indicators.loc[index, "ROC Lag"] > 0):
            spy_technical_indicators.loc[index, "ROC Evaluation"] = "ROC Falling Momentum"
    else:
        spy_technical_indicators.loc[index, "ROC Evaluation"] = "ROC Uninterpretable"
            
spy_technical_indicators.drop(columns = ["ROC Lag"], inplace = True)

#Creating Stochasic Indicator evaluation metrics
spy_technical_indicators["STO"] = TA.STOCH(spy_df)


spy_technical_indicators["Returns"] = spy_df["Return Direction"]
spy_technical_indicators.dropna(inplace = True)

spy_technical_indicators

,RSI,RSI Evaluation,CCI,CCI Evaluation,ROC,ROC Evaluation,STO,Returns
Date,,,,,,,,
2015-02-03,56.060269,No Abnormalities,76.640598,CCI Uptrend,1.546708,ROC Uninterpretable,83.095398,1.0
2015-02-04,53.981509,No Abnormalities,73.789992,CCI Potential Downtrend,2.121914,ROC Uninterpretable,73.808742,0.0
2015-02-05,58.371464,No Abnormalities,110.905553,CCI Potential Uptrend,1.883253,ROC Uninterpretable,97.867196,1.0
2015-02-06,56.758049,No Abnormalities,99.721561,CCI Potential Downtrend,-0.169990,ROC Falling Momentum,81.982833,0.0
2015-02-09,54.156271,No Abnormalities,50.893078,CCI Potential Downtrend,0.186060,ROC Uninterpretable,72.175065,0.0
...,...,...,...,...,...,...,...,...
2022-01-05,50.135922,No Abnormalities,11.190858,CCI Potential Downtrend,0.372752,ROC Uninterpretable,59.778060,0.0
2022-01-06,49.682393,No Abnormalities,-27.253761,CCI Downtrend,0.013632,ROC Uninterpretable,58.252392,0.0
2022-01-07,47.727484,No Abnormalities,-50.727289,CCI Potential Downtrend,0.295373,ROC Uninterpretable,51.837668,0.0


In [5]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder

categorical_variables = list(spy_technical_indicators.dtypes[spy_technical_indicators.dtypes == "object"].index)

categorical_variables
enc = OneHotEncoder(sparse= False)
encoded_data = enc.fit_transform(spy_technical_indicators[categorical_variables])

encoded_df = pd.DataFrame(encoded_data, columns = enc.get_feature_names(categorical_variables), index = spy_technical_indicators.index)

encoded_df = pd.concat([encoded_df, spy_technical_indicators.drop(columns = categorical_variables)], axis = 1)
encoded_df

,RSI Evaluation_No Abnormalities,RSI Evaluation_Overvalued,RSI Evaluation_RSI Bearish Signal,RSI Evaluation_RSI Bullish Signal,RSI Evaluation_Undervalued,CCI Evaluation_CCI Downtrend,CCI Evaluation_CCI Potential Downtrend,CCI Evaluation_CCI Potential Uptrend,CCI Evaluation_CCI Uptrend,ROC Evaluation_ROC Falling Momentum,ROC Evaluation_ROC Uninterpretable,RSI,CCI,ROC,STO,Returns
Date,,,,,,,,,,,,,,,,
2015-02-03,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,56.060269,76.640598,1.546708,83.095398,1.0
2015-02-04,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,53.981509,73.789992,2.121914,73.808742,0.0
2015-02-05,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,58.371464,110.905553,1.883253,97.867196,1.0
2015-02-06,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,56.758049,99.721561,-0.169990,81.982833,0.0
2015-02-09,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,54.156271,50.893078,0.186060,72.175065,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-05,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,50.135922,11.190858,0.372752,59.778060,0.0
2022-01-06,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,49.682393,-27.253761,0.013632,58.252392,0.0
2022-01-07,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,47.727484,-50.727289,0.295373,51.837668,0.0


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = encoded_df.drop(columns = ["Returns"])
y = encoded_df["Returns"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
neural = Sequential()

number_input_features = len(X.columns)
hidden_nodes_layer1 =  (number_input_features + 1) // 2 
hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) // 2

neural.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="elu"))
neural.add(Dense(units=hidden_nodes_layer2, activation="elu"))
neural.add(Dense(units=1, activation="sigmoid"))

neural.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 128       
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 169
Trainable params: 169
Non-trainable params: 0
_________________________________________________________________


In [8]:
neural.compile(loss = "binary_crossentropy", optimizer = "adam",  metrics = ["accuracy"])

model = neural.fit(X_train_scaled, y_train, epochs = 200)

Epoch 1/200
41/41 [==============================] - 0s 592us/step - loss: 0.5881 - accuracy: 0.7056
Epoch 2/200
41/41 [==============================] - 0s 598us/step - loss: 0.5306 - accuracy: 0.7597
Epoch 3/200
41/41 [==============================] - 0s 623us/step - loss: 0.4941 - accuracy: 0.7727
Epoch 4/200
41/41 [==============================] - 0s 823us/step - loss: 0.4654 - accuracy: 0.7941
Epoch 5/200
41/41 [==============================] - 0s 848us/step - loss: 0.4433 - accuracy: 0.8124
Epoch 6/200
41/41 [==============================] - 0s 848us/step - loss: 0.4259 - accuracy: 0.8253
Epoch 7/200
41/41 [==============================] - 0s 947us/step - loss: 0.4130 - accuracy: 0.8322
Epoch 8/200
41/41 [==============================] - 0s 873us/step - loss: 0.4050 - accuracy: 0.8284
Epoch 9/200
41/41 [==============================] - 0s 848us/step - loss: 0.3989 - accuracy: 0.8413
Epoch 10/200
41/41 [==============================] - 0s 873us/step - loss: 0.3940 - accura

In [16]:
Y_prediction = (neural.predict(X_train_scaled) > 0.5).astype("int32")
Y_prediction = Y_prediction.squeeze()

results = pd.DataFrame( {"Predictions": Y_prediction, "Actual": y_train})
display(results)

model_loss, model_accuracy = neural.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

,Predictions,Actual
Date,,
2020-10-20,0,1.0
2017-11-21,1,1.0
2015-07-13,1,1.0
2021-12-17,0,0.0
2017-04-24,1,1.0
...,...,...
2017-12-04,1,0.0
2018-09-06,0,0.0
2019-06-12,0,0.0


14/14 - 0s - loss: 0.4066 - accuracy: 0.8265 - 19ms/epoch - 1ms/step
Loss: 0.40660902857780457, Accuracy: 0.8264840245246887
